In [1]:
import os

# Replace this with the path to your main folder
root_folder = r"D:\Drive D\models"

# Get all files ending with "lr0001.h5" in all subfolders
model_paths = [
    os.path.join(dirpath, filename)
    for dirpath, dirnames, filenames in os.walk(root_folder)
    for filename in filenames
    if filename.endswith("lr0001.h5")
]

# print(f"There {len(model_paths)} models in {root_folder}")
# for file_path in model_paths:
#     print(file_path)

In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np

In [3]:
import os
from tensorflow.keras.preprocessing import image

def predict_images_in_folder(model, folder_path, image_size=(224, 224)):
    predictions = []
    
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.jpg')):
            img_path = os.path.join(folder_path, filename)
            
            # Load and preprocess the image
            img = image.load_img(img_path, target_size=image_size)
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            #img_array = img_array / 255.0  # Normalize the image
            
            # Make prediction
            prediction = model.predict(img_array, verbose=0)
            predictions.append(prediction[0])
            
            #print(f"Processed {filename}: {prediction[0]}")
    
    return predictions

# # Example for 1 folder prediction
# image_folder = r"D:\Drive D\data_x\test\erosion_0\52-11116 L"
# results = predict_images_in_folder(model, image_folder)
# for i, result in enumerate(results):
#     print(f"Image {i+1} prediction: {result}")

In [4]:
import os

def get_subfolder_paths(main_folder):
    subfolder_paths = []
    
    # Walk through the directory
    for root, dirs, files in os.walk(main_folder):
        for dir in dirs:
            # Construct full path of the subfolder
            subfolder_path = os.path.join(root, dir)
            subfolder_paths.append(subfolder_path)
    
    return subfolder_paths

# # Example usage
# main_folder = r"D:\Drive D\data_x\test\erosion_0"
# subfolders = get_subfolder_paths(main_folder)

# # Print all subfolder paths
# for subfolder in subfolders:
#     print(subfolder)

In [5]:
def consensus_testing(model_path) :
    model = load_model(model_path)
    model_name = os.path.basename(model_path)
    threshold = 0.5
    confident_val = 0.7

    main_folder = r"D:\Drive D\data_x\test\erosion_0"
    subfolders = get_subfolder_paths(main_folder)

    output_df = []

    for subfolder in subfolders:
        #print(f"Processing {subfolder}")
        results = predict_images_in_folder(model, subfolder)

        pos = [result for result in results if result > threshold + confident_val/2]
        neg = [result for result in results if result <= threshold - confident_val/2]

        if len(pos) >= len(neg) :
            predicted_class = 1
        else :
            predicted_class = 0
        
        patient_ID = subfolder.split(os.sep)[-1]

        output_df.append({
            'patient_ID': patient_ID,
            'pos' : len(pos),
            'neg' : len(neg),
            'real_class' : 0,
            'predicted_class' : predicted_class
        })

    main_folder = r"D:\Drive D\data_x\test\erosion_1"
    subfolders = get_subfolder_paths(main_folder)

    for subfolder in subfolders:
        #print(f"Processing {subfolder}")
        results = predict_images_in_folder(model, subfolder)

        pos = [result for result in results if result > threshold + confident_val/2]
        neg = [result for result in results if result <= threshold - confident_val/2]

        if len(pos) >= len(neg) :
            predicted_class = 1
        else :
            predicted_class = 0
    
        patient_ID = subfolder.split(os.sep)[-1]

        output_df.append({
            'patient_ID': patient_ID,
            'pos' : len(pos),
            'neg' : len(neg),
            'real_class' : 1,
            'predicted_class' : predicted_class
        })

    df_results = pd.DataFrame(output_df)

    save_folder = r"C:\Users\kanan\Desktop\Project_TMJOA\Phase_2.1\Consesus method"
    excel_filename = f"{model_name}_prediction_results.xlsx"
    save_path = os.path.join(save_folder,excel_filename)

    counter = 1
    while os.path.exists(save_path):
        excel_filename = f'{model_name}_prediction_results_{counter}.xlsx'
        save_path = os.path.join(save_folder,excel_filename)
        counter += 1

    df_results.to_excel(save_path, index=False)
    print(f"{excel_filename} saved to {save_path}")

    if 'real_class' not in df_results.columns or 'predicted_class' not in df_results.columns:
        raise ValueError("DataFrame must contain 'real_class' and 'predicted_class' columns")
    
    # Extract true labels and predictions
    y_true = df_results['real_class']
    y_pred = df_results['predicted_class']
    
    # Calculate True Positives, False Positives, False Negatives, True Negatives
    tp = ((y_true == 1) & (y_pred == 1)).sum()
    fp = ((y_true == 0) & (y_pred == 1)).sum()
    fn = ((y_true == 1) & (y_pred == 0)).sum()
    tn = ((y_true == 0) & (y_pred == 0)).sum()
    
    # Calculate metrics
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    # Return results as a dictionary
    return {
        'model_name' : model_name,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }

In [6]:
results = []

for model_path in model_paths:
    print(f"Testing : {model_path}")
    result = consensus_testing(model_path)
    results.append(result)

results_df = pd.DataFrame(results)
results_df

Testing : D:\Drive D\models\EfficientNetB5\EfficientNetB5_bo20_lr0001.h5
EfficientNetB5_bo20_lr0001.h5_prediction_results.xlsx saved to C:\Users\kanan\Desktop\Project_TMJOA\Phase_2.1\Consesus method\EfficientNetB5_bo20_lr0001.h5_prediction_results.xlsx
Testing : D:\Drive D\models\EfficientNetB7\EfficientNetB7_bo20_lr0001.h5
EfficientNetB7_bo20_lr0001.h5_prediction_results.xlsx saved to C:\Users\kanan\Desktop\Project_TMJOA\Phase_2.1\Consesus method\EfficientNetB7_bo20_lr0001.h5_prediction_results.xlsx
Testing : D:\Drive D\models\EfficientNetV2L\EfficientNetV2L_bo20_lr0001.h5
EfficientNetV2L_bo20_lr0001.h5_prediction_results.xlsx saved to C:\Users\kanan\Desktop\Project_TMJOA\Phase_2.1\Consesus method\EfficientNetV2L_bo20_lr0001.h5_prediction_results.xlsx
Testing : D:\Drive D\models\InceptionResNetV2\InceptionResNetV2_bo20_lr0001.h5
InceptionResNetV2_bo20_lr0001.h5_prediction_results.xlsx saved to C:\Users\kanan\Desktop\Project_TMJOA\Phase_2.1\Consesus method\InceptionResNetV2_bo20_lr0001

,model_name,accuracy,precision,recall,f1_score
0,EfficientNetB5_bo20_lr0001.h5,0.839286,0.906250,0.828571,0.865672
1,EfficientNetB7_bo20_lr0001.h5,0.839286,0.933333,0.800000,0.861538
2,EfficientNetV2L_bo20_lr0001.h5,0.839286,0.906250,0.828571,0.865672
3,InceptionResNetV2_bo20_lr0001.h5,0.625000,0.625000,1.000000,0.769231
4,InceptionV3_bo20_lr0001.h5,0.625000,0.625000,1.000000,0.769231
5,MobileNetV3Large_bo20_lr0001.h5,0.892857,0.939394,0.885714,0.911765
6,RegNetX320_bo20_lr0001.h5,0.928571,0.942857,0.942857,0.942857
7,RegNetY320_bo20_lr0001.h5,0.875000,0.937500,0.857143,0.895522
8,ResNet152-Bestof20_lr0001.h5,0.857143,0.935484,0.828571,0.878788
9,ResNet152V2_bo20_lr0001.h5,0.625000,0.625000,1.000000,0.769231


In [7]:
save_folder = r"C:\Users\kanan\Desktop\Project_TMJOA\Phase_2.1\Training logs"
excel_filename = r"consensus_model_evaluation_results.xlsx"
save_path = os.path.join(save_folder,excel_filename)

counter = 1
while os.path.exists(save_path):
    excel_filename = f'consensus_model_evaluation_results_{counter}.xlsx'
    save_path = os.path.join(save_folder,excel_filename)
    counter += 1

results_df.to_excel(save_path, index=False)
print(f"Results saved to {save_path}")

Results saved to C:\Users\kanan\Desktop\Project_TMJOA\Phase_2.1\Training logs\consensus_model_evaluation_results_1.xlsx
